In [1]:
# %pip install -U alpaca-py

In [2]:
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import CryptoHistoricalDataClient

# No keys required for crypto data
client = CryptoHistoricalDataClient()

In [3]:
from datetime import datetime

# Creating request object
request_params = CryptoBarsRequest(
  symbol_or_symbols=["BTC/USD"],
  timeframe=TimeFrame.Minute,
  start=datetime(2025, 1, 1),
  end=datetime(2025, 1, 31)
)

# Retrieve daily bars for Bitcoin in a DataFrame and printing it
btc = client.get_crypto_bars(request_params)

# Convert to dataframe
btc = btc.df.reset_index(level=['symbol'])

btc

,symbol,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2025-01-01 00:00:00+00:00,BTC/USD,93381.5825,93451.9400,93381.5825,93399.3030,0.000062,1.0,93451.9400
2025-01-01 00:01:00+00:00,BTC/USD,93469.6285,93469.6285,93469.6285,93469.6285,0.000000,0.0,93469.6285
2025-01-01 00:02:00+00:00,BTC/USD,93501.8005,93501.8005,93501.8005,93501.8005,0.000000,0.0,93501.8005
2025-01-01 00:03:00+00:00,BTC/USD,93461.5430,93461.5430,93461.5430,93461.5430,0.000000,0.0,93461.5430
2025-01-01 00:06:00+00:00,BTC/USD,93405.7060,93405.7060,93405.7060,93405.7060,0.000000,0.0,93405.7060
...,...,...,...,...,...,...,...,...
2025-01-22 01:13:00+00:00,BTC/USD,105810.5050,105810.6920,105810.5050,105810.6920,0.000000,0.0,105810.5985
2025-01-22 01:14:00+00:00,BTC/USD,105864.8500,105864.8500,105864.8500,105864.8500,0.000000,0.0,105864.8500
2025-01-22 01:15:00+00:00,BTC/USD,105922.8100,105922.8100,105881.3900,105881.3900,0.000000,0.0,105902.1000


In [ ]:
import plotly.express as px

px.line(btc, y=["low", "high"])

In [ ]:
import numpy as np
import numba
import torch

fee = 0.003
p_sell = torch.tensor(btc['low'].values).float() * (1 - fee)
p_buy = torch.tensor(btc['high'].values).float() * (1 + fee)


@numba.jit
def best_holdings(p_buy, p_sell):
    n = p_buy.size
    stock = np.zeros(n)
    cash = np.zeros(n)
    cash_from_stocks = np.zeros(n, dtype=np.bool)
    stock_from_stocks = np.zeros(n, dtype=np.bool)

    stock[0] = -p_buy[0]
    for i in range(1, n):
        cash[i] = max(cash[i-1], stock[i-1] + p_sell[i])
        cash_from_stocks[i] = cash[i-1] < stock[i-1] + p_sell[i]
        stock[i] = max(cash[i-1] - p_buy[i], stock[i-1])
        stock_from_stocks[i] = cash[i-1] - p_buy[i] < stock[i-1]

    holding = np.zeros(n, dtype=np.bool)
    for i in range(n-1, 0, -1):
        if holding[i]:
            holding[i-1] = stock_from_stocks[i]
        else:
            holding[i-1] = cash_from_stocks[i]
    return holding, cash

holding, cash = best_holdings(p_buy.numpy(), p_sell.numpy())
n_trades = np.sum(holding[1:] != holding[:-1])

n_trades, cash[-1]

(np.int64(220), np.float64(100211.75))

In [6]:
import torch

class Model(torch.nn.Module):
    def __init__(self, n=64):
        super().__init__()
        self.n = n
        self.drop = torch.nn.Dropout(p=0.3)
        self.li = torch.nn.Sequential(
            torch.nn.Conv1d(2, n, n),
            torch.nn.Softplus(),
            torch.nn.Conv1d(n, n, 2),
            torch.nn.Softplus(),
            torch.nn.Conv1d(n, n, 2, dilation=2),
            torch.nn.Softplus(),
            torch.nn.Conv1d(n, 1, 2, dilation=4),
        )

    def forward(self, x):
        n = x.shape[-1]
        x = x.log()
        x = x - x.mean()
        x = self.drop(x)
        x = self.li(x)
        x = x - x.mean()
        # x = self.drop(x)
        return x.flatten(-2)


model = Model(64).cuda()

p = torch.stack((p_sell, p_buy), dim=0)[None,...].cuda()

# y = model(p)
# print(f'{p.shape=}')
# print(f'{y.shape=}')



In [ ]:
from sqrll.sqrll import sqrll_kernel

with torch.no_grad():
    # print(f'{p.shape=}')
    y = model(p)[:, :-2, None]
    # print(f'{y.shape=}')
    y = torch.nn.functional.pad(y, (0,0,1,1), value=-10)
    # print(f'{y.shape=}')

x = (y > 0) * 0.8 + 0.1
r = y.tanh().abs()
pos = sqrll_kernel(x * r, 1 - r)[...,0]
cost = pos[:,1:] - pos[:,:-1]
# print(f'{cost.shape=}')
which = (y > 0).transpose(1,2).long()[...,1:]
# print(f'{which.shape=}')
pc = p[...,-cost.shape[-1]:].gather(1, which)
cost = cost * pc
# print(f'{pc.shape=}')
cost = cost.cumsum(dim=-1)
px.line(cost.cpu().flatten())

In [13]:
from sqrll.sqrll import sqrll_kernel
import numpy as np
from tqdm import tqdm

opt = torch.optim.AdamW(
    model.parameters(),
    lr=1e-3,
    betas=(0.9, 0.999),
    weight_decay=0,
)

# loss_fn = torch.nn.BCEWithLogitsLoss(reduction='none')

# target = (x[1:] > x[:-1]).float()
# weight = (x[1:].log() - x[:-1].log()).abs()
# target = torch.tensor(holding).float().cuda()


for _ in (prog := tqdm(range(5000))):
    opt.zero_grad()
    
    y = model(p)[:, :-2, None]
    # print(f'{y.shape=}')
    y = torch.nn.functional.pad(y, (0,0,1,1), value=-10)
    # print(f'{y.shape=}')

    x = y > 0
    r = y.tanh().abs()
    pos = sqrll_kernel(x * r, 1 - r)[...,0]
    cost = pos[:,1:] - pos[:,:-1]
    # print(f'{cost.shape=}')
    which = x.transpose(1,2).long()[...,1:]
    # print(f'{which.shape=}')
    pc = p[...,-cost.shape[-1]:].gather(1, which)
    cost = cost * pc

    loss = cost.sum()
    
    prog.set_description(f'{loss.item():.4f}')
    loss.backward()
    opt.step()

loss.item()

-11728.4648: 100%|██████████| 5000/5000 [00:17<00:00, 288.48it/s]


-11728.46484375

In [ ]:
import pandas as pd

with torch.no_grad():
    y = model(p[...,:])[0, :-1].sigmoid()
    # y = torch.nn.functional.pad(y[:, :-2], (1,1))
    # buy = y[:, 1:] - y[:, :-1]
    # which = (buy>0)[:,None,:].long()
    # print(f'{which.shape=}')
    # price = p_fee[..., 1:].gather(1, which)
    # price = p_fee[..., which, 1:]
    # print(f'{price.shape=}')
    # buy = y[:, :-2] - y[:, 1:-1]

# px.line(loss_fn(torch.zeros_like(target), target).cpu().flatten())
px.line(y.cpu().flatten())
# df = pd.DataFrame({
#     "y": y.cpu().flatten(),
#     "loss": loss_all.detach().cpu().flatten()
#     })
# px.line(df, y=['y', 'loss'])
# px.line({
#     "y": y.cpu().flatten(),
#     "loss": loss_all.detach().cpu().flatten()
# })
# px.line(loss_all.detach().cpu().flatten())
# px.line((buy * price).cpu().flatten())
# px.line(p[0,0].log().cpu().flatten())
# (buy * price).sum()

In [10]:
import numpy as np

fee = 0.01
p_fee = p.cpu() * torch.tensor([[1-fee], [1+fee]])

n = p_fee.shape[-1]
stock = np.zeros(n)
cash = np.zeros(n)
cash_src = np.zeros(n)
stock_src = np.zeros(n)

stock[0] = -p_fee[0,1,0]
for i in range(1, n):
    cash[i] = max(cash[i-1], stock[i-1] + p_fee[0,0,i])
    cash_src[i] = cash[i-1] < stock[i-1] + p_fee[0,0,i]
    stock[i] = max(cash[i-1] - p_fee[0,1,i], stock[i-1])
    stock_src[i] = cash[i-1] - p_fee[0,1,i] < stock[i-1]

best = np.zeros(n)
best[-1] = 0
for i in range(n-1, 0, -1):
    if best[i] == 0:
        best[i-1] = cash_src[i]
    else:
        best[i-1] = stock_src[i]

trades = np.sum(best[1:] != best[:-1])

cash[-1], trades

(np.float64(35566.7109375), np.int64(20))

In [11]:
import numba
import numpy as np

@numba.jit
def trade_forward(trade, p_buy, p_sell):
    n = trade.shape[-1]
    cash = np.zeros_like(trade)
    cash[:, 0] = 1
    for i in range(1, n):
        if trade[i-1] > 0
            cash[:, i] *= 1 - trade[i-1]
        else:
            cash[:, i] *= 
    
t = model(p)
l = trade_forward(t, p_buy, p_sell)


# class TradeLoss(torch.autograd.Function):

#     @staticmethod
#     def forward(ctx, trade, p_buy, p_sell):
#         loss = trade_forward(
#             trade.cpu().numpy(), 
#             p_buy.cpu().numpy(),
#             p_sell.cpu().numpy()
#         )

SyntaxError: expected ':' (2559706096.py, line 10)

In [ ]:
px.line(cash)

In [ ]:
px.line(holding)

In [ ]:
yhold = y.cpu().flatten() > 0.5
yp = p[0, :, -yhold.shape[-1]:]

for i in range()